# Finding Waldo

See ```image_processing_process.ipynb``` for details of how we preprocessed the images via chopping and cropping.

## 0. Installing and importing dependencies

In [ ]:
import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn ## Pre-trained model from PyTorch
from torchvision.transforms import functional as F
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import os
import shutil

import config
from xml_to_csv import convert_each_xml_to_csv
from bbox_regressor import ObjectDetector
from custom_tensor_dataset import CustomTensorDataset

import os
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
from image_processing import load_annotations

Go to ```config.py``` and ensure your data paths are properly initialized. Then check with this code block.

In [3]:
print(f"Base path: {config.BASE_PATH}")
print(f"Annotation path: {config.ANNOTS_PATH}")
print(f"Images path: {config.IMAGES_PATH}")
print(f"Ouput path: {config.BASE_OUTPUT}")

Base path: /Users/laurenbeede/FindWaldo
Annotation path: /Users/laurenbeede/FindWaldo/annotations
Images path: /Users/laurenbeede/FindWaldo
Ouput path: /Users/laurenbeede/FindWaldo/output


## 1. Data Exploration

In [ ]:
# Defining the current path to GitHub repo
datapath = os.getcwd()

# Check how many images of Waldo and not Waldo are in the dataset
numnotwaldo = len(os.listdir(datapath+'/chopped-256/notwaldo/'))
numwaldo = len(os.listdir(datapath+'/chopped-256/waldo/'))

print(f"There are {numwaldo} images of Waldo")
print(f"There are {numnotwaldo} images of not Waldo")

There are 18 images of Waldo
There are 270 images of not Waldo


Let's plot the original images of the whole scene and place the corresponding bounding box atop Waldo's location.

In [ ]:
# Setting the directory paths
annotation_dir = os.getcwd() + '/annotations/bnd_box'  # Folder containing XML files from Kenstler
images_folder = os.getcwd() + '/annotations/original_imgs'       # Folder containing original images from Kenstler

# Loading the annotations from XML files
annotations = load_annotations(annotation_dir)

# Itearting over the annotations and plotting each image with its bounding box
for filename, data in annotations.items():
    image_path = os.path.join(images_folder, filename)
    if not os.path.exists(image_path):
        print(f"Image {filename} not found.")
        continue

    try:
        img = Image.open(image_path)
    except Exception as e:
        print(f"Could not open image {filename}: {e}")
        continue

    # Set up the plot.
    fig, ax = plt.subplots(1)
    ax.imshow(img)

    bbox = data.get("bbox")
    if bbox:
        xmin, ymin, xmax, ymax = bbox
        width = xmax - xmin
        height = ymax - ymin

        # Create a rectangle patch with red edges.
        rect = patches.Rectangle((xmin, ymin), width, height,
                                 linewidth=2, edgecolor='red', facecolor='none')
        ax.add_patch(rect)

    ax.set_title(filename)
    plt.show()

## 2. Training

In [ ]:
# Runs the training script
# This will train the model using the dataset and save the trained model to the specified path

%run train.py

## 3. Predicting

In [ ]:
# Runs the prediction script
# This will use the trained model to predict the bounding boxes for the images in the test set
# Takes in one argument: the path to the test set
# The test set is specified in the test_paths.txt file as an output from the train.py script

%run predict.py -i /path/to/FindWaldo/output/test_paths.txt